# Exploring Foursquare
Updated, again, 05/31/19
* How much data can I get in a day?
* What categories should I examine and how?
* Will "shrinking box" exploration work?
* Does Folium have a square marker (try in other notebook?)

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
# @hidden cell
# Foursquare information

CLIENT_ID = 'CWCPNAVXDH3TI1BGS4VED4ANSUKEFGHBA4511GRPYPKPNJRD' # your Foursquare ID
CLIENT_SECRET = 'ALMFEYIENSPH3RV3TQB1NGWTKTANVJ5QTHAVZ5B1GRJWIP21' # your Foursquare Secret

In [0]:
# Foursquare query information that will be constant during this project
VERSION = '20190427' # Foursquare API version
LIMIT = 120

In [0]:
# define function that extracts the category of the venue
# used in Foursquare section

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
# define function to extract Foursquare information given
# a bounding box and a list of categories

def getVenuesByBox(sw, ne,    # the southwest & northeast box corners, tuples
                   codes      # Foursquare venue codes, list of strings
                  ):
  
  # Foursquare category ID requires comma-separated text string
  code_string = ",".join(codes)
  
  # define the search URL
  url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}' + \
      f'&client_secret={CLIENT_SECRET}&v={VERSION}&sw={sw[0]},{sw[1]}' + \
      f'&ne={ne[0]},{ne[1]}' + \
      f'&categoryId={code_string}&limit={LIMIT}'
  
  # submit the url and capture the returned text
  venues = requests.get(url).json()["response"]['groups'][0]['items']
  
  # if any venues are captured, process and return dataframe. Otherwise, return None. 
  if venues:
  
    venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.postalCode', 
                        'venue.location.lat', 'venue.location.lng', 'venue.location.state']
    venues = venues.loc[:, filtered_columns]

    # filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis=1)

    # clean columns
    venues.columns = [col.split(".")[-1] for col in venues.columns]
  
    return venues
  
  else:
    
    return None  # no venues in box meet criteria
  
  

In [0]:
# Create dataframe to hold areas and number of clients (venues) in each

df_box = pd.DataFrame(columns=['lat','lng','clients'])


In [0]:
# Define function to split a bounding box into 4 sub-boxes

def split_box(sw, ne):
  len_x, len_y = (ne[0] - sw[0])/2, (ne[1] - sw[1])/2
  sw_corners = [sw,
                (sw[0]+len_x,sw[1]),
                (sw[0],sw[1]+len_y),
                (sw[0]+len_x,sw[1]+len_y)]
  boxes = []
  
  for corner in sw_corners:
    boxes.append((corner, (corner[0]+len_x, corner[1]+len_y)))
    
  return boxes

In [0]:
def shrink_box(sw, ne, codes):
  df_results = getVenuesByBox(sw, ne, codes)
  
  if df_results:
    
    if df_results[0] < 100:
      
      return df_results
    
    else:    
      # Check for size, recurse & append if necessary
    
      # divide box into 4 pieces and recurse
      
       
      
    
      return df_results
  
  else:
    return "popcorn!"
    
  
  

# Shrinking Box Pseudo Code


        
    def shrink(sw, ne, codes)
        df_results = query 4square(sw, ne, codes)
        
        calculate num_clients_ga
        
        if num_clients_ga = 0:
            add current box (sw, ne, num_clients_ga) to df_box
            return
            
        if number of results <100:
            drop clients not in ga
            add current box (sw, ne, num_clients_ga) to df_box
            append Georgia clients to df_client
            return
        
        # case where number of raw results >=100 and at least 1 in GA
        location_set = define 4 tupples of sw&ne locations 
        for box in location_set:
            shrink(box[0, box[1], codes)
            
        return

        
        
        
        

In [7]:
asian = ['4bf58dd8d48988d142941735']
indian = ['4bf58dd8d48988d10f941735']

asian = indian + asian

fsw = (33.946964, -84.150187)  
fne = (33.971041, -84.121381)

# fsw = (31.468720, -79.166785)  # Offshore - used to test for area with no venues
# fne = (31.662861, -78.990998)





# df_asian = 
# df_indian = shrink_box(fsw, fne, indian)
df_nearme = getVenuesByBox(fsw, fne, asian)
print(df_nearme.shape)

df_nearme.head()

(100, 6)


,name,categories,postalCode,lat,lng,state
0,Saigon Cafe Duluth,Asian Restaurant,30096,33.960122,-84.136158,GA
1,Seo Ra Beol Restaurant,Asian Restaurant,30096,33.963876,-84.141964,GA
2,Mini Hot Pot 2,Asian Restaurant,30096,33.959822,-84.134848,GA
3,SunO Dessert,Asian Restaurant,30096,33.969180,-84.145040,GA
4,Assi Plaza,Supermarket,30096,33.947160,-84.123698,GA


In [0]:
print(df_nearme.shape)
print(set(df_nearme['state']))
df_indian.head()


(58, 6)
{'GA', 'Georgia'}


,name,categories,postalCode,lat,lng,state
0,Curry Cuisine Indian Food,Indian Restaurant,NaN,33.960464,-84.12516,GA


In [0]:
# Location for sample query - basically my neighborhood in Duluth

super_h = (33.969303, -84.143626)

# corners for Foursquare bounding box search


albany_ne = (32.114878, -83.489320)
albany_sw = (31.517953, -84.222243)

# Foursquare categories

# Korean restaurant (sub category of above) '4bf58dd8d48988d113941735'
# Indian restaurant (not sub category of above) '4bf58dd8d48988d10f941735'

In [0]:
# Define search url and test.

# fsw, fne = albany_sw, albany_ne



url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}' + \
      f'&client_secret={CLIENT_SECRET}&v={VERSION}&sw={fsw[0]},{fsw[1]}' + \
      f'&ne={fne[0]},{fne[1]}' + \
      f'&categoryId={asian}' # &limit={LIMIT}'    

results = requests.get(url).json()

In [0]:
# Now convert to dataframe and examine. 

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.postalCode','venue.location.lat','venue.location.lng', 'venue.location.state']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.sort_values(by=['categories']).head(50)

KeyError: ignored

In [0]:
print(venues)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b2d1a43f964a52056ce24e3', 'name': 'Saigon Cafe Duluth', 'location': {'address': '3675 Satellite Blvd #750', 'crossStreet': 'and Pleasant Hill Rd', 'lat': 33.96012236013812, 'lng': -84.13615842313993, 'labeledLatLngs': [{'label': 'display', 'lat': 33.96012236013812, 'lng': -84.13615842313993}], 'postalCode': '30096', 'cc': 'US', 'city': 'Duluth', 'state': 'GA', 'country': 'United States', 'formattedAddress': ['3675 Satellite Blvd #750 (and Pleasant Hill Rd)', 'Duluth, GA 30096', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d142941735', 'name': 'Asian Restaurant', 'pluralName': 'Asian Restaurants', 'shortName': 'Asian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b2d1a43f964a52056ce24e3-0'}, {'reasons':

# Data structures needed

    df_box: 
    - sw, ne
    - number of clients
    
    df_client: 
    - Name
    - lat & long
    - categories
    - postal code (?)
    - state

# Foursquare Information

Free plan includes:
* 2 queries per second
* 95,000 regular calls per day
* 500 premium calls per day

Attribution is **required**: Whenever Foursquare data is showing on the screen, attribution to Foursquare must be visible simultaneously or before the Foursquare data is shown.



   